In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

In [86]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [43]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [44]:
test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [55]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [57]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [58]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [59]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [60]:
#target variable
y_train = train['Target']
test_id = test['Id']

In [61]:
#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [83]:
#train model
clf = RandomForestClassifier(n_estimators=500,max_features=3,min_samples_split=5,oob_score=True)
clf.fit(train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=5, min_weight_fraction_leaf=0.0,
            n_estimators=500, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [84]:
#predict on test data
pred = clf.predict_proba(test)

In [97]:
#write submission file and submit
columns = ['Front','Left','Rear','Right']
sub = pd.DataFrame(data=pred, columns=columns)
sub['Id'] = test_id
sub = sub[['Id','Front','Left','Rear','Right']]
sub.to_csv("sub_rfih.csv", index=False) #99.8XXX